In [33]:
import os
from datetime import datetime as dt, timedelta

import pandas as pd

In [8]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../gcp-credentials.json"
PROJECT_ID = "digitaltwin-49d59"
PROJECT_NUMBER = "723647330842"
SECRET_ID = "oura-personal-access-token"

In [58]:
import json
from enum import Enum


class JsonlineClient:
    def __init__(self, dir_path: str) -> None:
        self.dir_path = dir_path

    def insert(self, json_list: list, file_name):
        file_path = f'{self.dir_path}/{file_name}.jsonl'
        with open(file_path, 'a', encoding='utf8') as f:
            for dic in json_list:
                json.dump(dic, f, ensure_ascii=False)
                f.write('\n')

In [91]:
from google.cloud import bigquery


def insert(json_list, table_name):
    bq_client = bigquery.Client()
    table_id = f"digitaltwin-49d59.oura.{table_name}"

    errors = bq_client.insert_rows_json(table_id, json_list)
    if errors != []:
        print("Encountered errors while inserting rows: {}".format(errors))

In [19]:
def fetch_oura_token():
    from google.cloud import secretmanager
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(name=f'projects/{PROJECT_NUMBER}/secrets/{SECRET_ID}/versions/latest')
    return response.payload.data

In [20]:
OURA_TOKEN = fetch_oura_token()

In [23]:
from oura import OuraClient
client = OuraClient(personal_access_token=OURA_TOKEN)
who_am_i = client.user_info()
who_am_i

{'age': 24,
 'weight': 56.0,
 'height': 169,
 'gender': 'male',
 'email': 'makeffort134@gmail.com'}

### activityを例にデータ取得、テーブル作成、クエリして最新を取得、最新をstartにしてAPIリクエストまで

In [38]:
# 一旦データ取得・可視化を優先してライブラリを使用する。APIはシンプルなのでしばらく耐えると思う。
import oura
oura_client = oura.client_pandas.OuraClient(personal_access_token=OURA_TOKEN)

In [79]:
# 一旦めっちゃ昔からすべてのデータ構造パターンが返りそうな量を取得
res = oura_client.activity_summary(start=str(dt.today() - timedelta(days=2100)))

In [80]:
len(res['activity'])

627

In [82]:
# BQのテーブル作成をschemaありで行うためにjsonlに出力
JsonlineClient('../data').insert(res['activity'], 'acitivity')

In [133]:
# 上の出力ファイルからactivityテーブルをBQで手動で作成
def fetch_latest_date(table_name, column_date='summary_date'):
    client = bigquery.Client()

    query = f"""
        SELECT 
            {column_date}
        FROM 
            `digitaltwin-49d59.oura.{table_name}` 
        ORDER BY {column_date} DESC
        LIMIT 1
        """

    query_job = client.query(query).result()

    return list(query_job)[0][column_date]
    
last_date = fetch_latest_date(table_name='activity')
last_date

datetime.date(2021, 9, 23)

In [124]:
def fetch_and_update_oura_activity():
    summary_kind = 'activity'
    latest_date = fetch_latest_date(summary_kind)
    res = oura_client.activity_summary(start=str(latest_date + timedelta(days=1)))
    print(f"data size to insert BQ: {len(res[summary_kind])}")
    if len(res[summary_kind]):
        insert(res[summary_kind], summary_kind)

In [109]:
fetch_and_update_oura_activity()

data size to insert BQ: 0


### activity以外もテーブル作成

In [112]:
res = oura_client.bedtime_summary(start=str(dt.today() - timedelta(days=2100)))
# res = oura_client.bedtime_summary(start=str(dt.today() - timedelta(days=20)))
JsonlineClient('../data').insert(res['ideal_bedtimes'], 'bedtime')

In [116]:
res = oura_client.readiness_summary(start=str(dt.today() - timedelta(days=2100)))
# res = oura_client.readiness_summary(start=str(dt.today() - timedelta(days=20)))
JsonlineClient('../data').insert(res['readiness'], 'readiness')

In [119]:
res = oura_client.sleep_summary(start=str(dt.today() - timedelta(days=2100)))
# res = oura_client.sleep_summary(start=str(dt.today() - timedelta(days=20)))
JsonlineClient('../data').insert(res['sleep'], 'sleep')

### マスターとなる関数を作成

In [129]:
def fetch_and_update_oura_bedtime():
    latest_date = fetch_latest_date('bedtime', column_date='date')
    res_key = res_key_dict['bedtime']
    res = oura_client.bedtime_summary(start=str(latest_date + timedelta(days=1)))
    jsonl = res['ideal_bedtimes']
    print(f"bedtime: data size to insert BQ: {len(jsonl)}")
    if len(jsonl):
        insert(jsonl, summary_kind)


def fetch_and_update_oura():
    for summary_kind in ['activity', 'sleep', 'readiness']:
        latest_date = fetch_latest_date(summary_kind)
        res = getattr(oura_client, f'{summary_kind}_summary')(start=str(latest_date + timedelta(days=1)))
        jsonl = res[summary_kind]
        print(f"{summary_kind}: data size to insert BQ: {len(jsonl)}")
        if len(jsonl):
            insert(jsonl, summary_kind)
    fetch_and_update_oura_bedtime()

In [135]:
fetch_and_update_oura()

activity: data size to insert BQ: 0
sleep: data size to insert BQ: 0
readiness: data size to insert BQ: 0
bedtime: data size to insert BQ: 0
